In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load dataset
df = pd.read_csv(r"E:\github\data science\data-science\deep learning\sonar.csv")

# Prepare dataset
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Encode labels (R=0, M=1)
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Convert to tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
num_features = X.shape[1]

# ----- Single Layer Perceptron -----
class SingleLayerPerceptron(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

# Initialize model
model = SingleLayerPerceptron(num_features)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training loop
epochs = 100
for epoch in range(epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluate accuracy
with torch.no_grad():
    y_pred_test = model(X_test)
    y_pred_classes = (y_pred_test > 0.5).float()
    acc = (y_pred_classes.eq(y_test).sum() / float(y_test.shape[0])).item()
    print(f"Test Accuracy: {acc*100:.2f}%")

# ----- Take manual input -----
print("\nEnter sonar feature values (60 numbers).")
print("For quick test, you can enter first 10 values and the rest will be 0.")

features = []
for i in range(num_features):
    if i < 10:   # ask for first 10 values only
        val = float(input(f"Enter feature {i+1}: "))
    else:
        val = 0.0   # fill rest with 0 for simplicity
    features.append(val)

sample = torch.tensor([features], dtype=torch.float32)

with torch.no_grad():
    prediction = model(sample)
    predicted_class = (prediction > 0.5).int().item()
    print("Predicted Output:", "Mine" if predicted_class == 1 else "Rock")


Epoch [10/100], Loss: 0.6620
Epoch [20/100], Loss: 0.6526
Epoch [30/100], Loss: 0.6441
Epoch [40/100], Loss: 0.6363
Epoch [50/100], Loss: 0.6290
Epoch [60/100], Loss: 0.6222
Epoch [70/100], Loss: 0.6159
Epoch [80/100], Loss: 0.6099
Epoch [90/100], Loss: 0.6042
Epoch [100/100], Loss: 0.5988
Test Accuracy: 71.43%

Enter sonar feature values (60 numbers).
For quick test, you can enter first 10 values and the rest will be 0.
Predicted Output: Rock
